# Sensitivity Analysis

In [ ]:
from pyquantification.experiments import cached_experiments
from pyquantification.evaluation import (
    display_dataset_summary,
    plot_bin_sensitivity,
)

In [ ]:
quantification_methods = [
    'gsls',
]
dataset_labels = {
    'insect-sex': 'ISX-500',
    'insect-sex_smaller': 'ISX-250',
    'insect-sex_smallest': 'ISX-50',
}
common_options = dict(
    dataset_names=list(dataset_labels.keys()),
    classifier_names=['logreg'],
    calibration_methods=['uncalibrated'],
    loss_weights=[0, 0.3, 0.7, 1],
    gain_weights=[0, 0.3, 0.7, 1],
    random_states=list(range(0, 100)),
    shift_types=['gsls_shift'],
    bin_counts=[5, 'auto', 50],
    random_priors_options=[True],
)
# Run classifiers first with parallelism, as they do not affect the runtime.
cached_experiments(
    cache_key='classifier_only_bins_results',
    **common_options,
    quantification_methods=[],
    classification_workers=10,
    continue_on_failure=True,
)
results_df = cached_experiments(
    cache_key='bins_results',
    **common_options,
    quantification_methods=quantification_methods,
    classification_workers=1,
    continue_on_failure=True,
)
# Also evaluate speed of PCC and EM under the same conditions
quantification_runtime_df = cached_experiments(
    cache_key='quantification_runtime_results',
    **{**common_options, 'bin_counts': ['auto']},
    quantification_methods=['gsls', 'pcc', 'em'],
    classification_workers=1,
    continue_on_failure=True,
)

In [ ]:
# Allow for initial filtering of results
plot_df = results_df.reset_index()

# Set dataset labels
plot_df['dataset_label'] = plot_df['dataset_name'].map(dataset_labels)

## Dataset Summary

In [ ]:
display_dataset_summary(dataset_labels)

## Bin/Instance Count Sensitivity

In [ ]:
fig = plot_bin_sensitivity(
    plot_df,
    dataset_label_colors={
        'ISX-500': '#332288',
        'ISX-250': '#828CD6',
        'ISX-50': '#DAFAEF',
    },
    bin_count_symbols={
        5: ('triangle-down', 30),
        'auto': ('circle', 24),
        50: ('triangle-up', 30),
    },
)
fig.write_image("plots/bin-sensitivity.svg")
fig.show()